# Calculation of uncertanties for the ellipse fiting routine

Currently set for ZIMPOL data but can be used for IRDIS with small modification

In [1]:
#fitting ellipse to rings
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
from astropy.io import fits
import os, fnmatch
import matplotlib.gridspec as gridspec
from scipy import optimize
from skimage.measure import EllipseModel
from scipy import interpolate
from textwrap import wrap
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math

import functions as f


#comments
#central ellipse is fitted using gaussian. Gaussian is fitted in coordinates (-n/2, n/2) thats why at some point I add n/2 to correct for it and move ellipse points coordinates back to array coordinates. For iras15 second ring is also fitted with gaussian. Other arcs are fitted with an usual ellipse fitting routine (just to coordinates of pixels)
def boot_answer(boot_sample):
    err=np.std(boot_sample)
    mean=sum(boot_sample)/len(boot_sample)
    boot_median_CI_95 = np.percentile(boot_sample, [2.3,97.7])
    boot_median_CI_68 = np.percentile(boot_sample, [16,84])
    return mean,err,boot_median_CI_68,boot_median_CI_95   

def bootstrap_ellipse_fit(data,iterations):
    boot_sample_xc = []
    boot_sample_yc = []
    boot_sample_a = []
    boot_sample_b = []
    boot_sample_theta = []
    boot_sample_cosi = []

    for i in range(iterations):
        ell=EllipseModel()
        boot_sample = np.random.choice(len(data[:,0]),replace = True, size = len(data[:,0]))
        x_boot=data[boot_sample,0]
        y_boot=data[boot_sample,1]
        boot_sample_data=data*0.
        boot_sample_data[:,0]=x_boot
        boot_sample_data[:,1]=y_boot
        try:
            ell.estimate(boot_sample_data)
            xc_b, yc_b, a_b, b_b, theta_b= ell.params
            if a_b<b_b: a_b, b_b=b_b, a_b
            cosi_b=b_b/a_b
        except TypeError:
            break
            
        boot_sample_xc.append(xc_b)
        boot_sample_yc.append(yc_b)
        boot_sample_a.append(a_b)
        boot_sample_b.append(b_b)
        boot_sample_theta.append(theta_b)
        boot_sample_cosi.append(cosi_b)
  
    boot={'xc':boot_answer(boot_sample_xc),'yc':boot_answer(boot_sample_yc),'a':boot_answer(boot_sample_a),'b':boot_answer(boot_sample_b),'theta':boot_answer(boot_sample_theta),'cosi':boot_answer(boot_sample_cosi)}
           
    return boot

def LoadImage (dir,annulus):

    files = os.listdir(dir)
    for fil in files:
        if fnmatch.fnmatch(fil,  '*'+annulus+'_decon.fits'):
            
            hdu = fits.open(dir+fil)
            img = hdu[0].data
            ps =3.6
            n_in = img.shape[0]
            adc=int(400)
            bdc=int(n_in-400)
            image= img[adc:bdc,adc:bdc]
            n=image.shape[0]
            
            d=(n-1)/2
            x = np.linspace(-d, d, n)
            y = np.linspace(-d, d, n)
            x2, y2 = np.meshgrid(x, y)
            R = np.sqrt(x2**2+y2**2)
    return image, R, x, y, d

def giveR(min=1e-5, max=12, n=300):
    return np.linspace(1e-5, 12, n)

def sampledir(angle, n):
    xd, yd = [], []
    R = giveR()
    for r in R:
        xd.append( r*np.cos(angle*np.pi/180.) )
        yd.append( r*np.sin(angle*np.pi/180.) )
    return np.array(xd), np.array(yd)

def giveProfile(img, x, y, angle):

    fimg = interpolate.RectBivariateSpline(x, y, img)
    xd, yd = sampledir(angle, 100)
    profile = []
    for xx, yy in zip(xd, yd):
        profile.append(fimg(xx, yy)[0][0])
    #fig, ax = plt.subplots()
    #plt.plot(giveR(), profile, '.')
    return np.array(profile)
    
def Gauss1D(R, amp, r0, size):
    model = amp * np.exp( -(R-r0)**2 / size )
    return model

def fitProfile(profile, r):
    amp = amp_dict[star]
    r0 = r0_dict[star]
    size = size_dict[star]
    if second_ring:  #for second ring of iras15
        amp = 20
        r0 = 10
        size =0.7
    a = [amp, r0, size]
    sigma = 1 / (profile**2 +1e-10)
    try: 
        popt, pcov = optimize.curve_fit( Gauss1D, giveR(), profile, p0=a, sigma=sigma, maxfev=10000)
    except:
        popt=a
        popt[0]=2.5e-4

    #plt.plot(giveR(), Gauss1D(giveR(), *popt))
    #plt.show()
    #plt.close()

    return popt[1], popt[0]

def GetRingCoord(img, x, y, nangles = 360):
    ## Gives the radius, PA, amplitude of the fitte gaussian to nangles angles with a center chose to be (x,y)
    angles = np.linspace(0, 360, nangles)
    radii, amps = [], []
    for ang in angles:
        profile = giveProfile(img, x, y, ang)
        rad, amp = fitProfile(profile, giveR())
        radii.append(rad)
        amps.append(amp)
    #print(profile)
    return np.array(radii), 90-np.array(angles), np.array(amps)

def gauss_ell_fit(img,x,y,ps,savefig1, star,annulus,Rlimit):
    n=img.shape[0]
    shift=n/2-0.5

    rad, theta, amps = GetRingCoord(img, x, y, nangles = 360)

    d=n*ps/2
    fig, ax = plt.subplots()
    plt.imshow(np.arcsinh(img), extent=[-d, d, d, -d])
    plt.xlim(-lim * ps, lim * ps)
    plt.ylim(-lim * ps, lim * ps)
    plt.colorbar()
    xring = rad*np.cos(theta*np.pi/180.)
    yring = rad*np.sin(theta*np.pi/180.)

    #print(amps)

    xrok, yrok = [], []

    for xr, yr, amp in zip(xring, yring, amps):
        Rr = np.sqrt(xr**2+yr**2)
        if Rr<Rlimit and second_ring==False:
            if amp > 2.5e-4:
                plt.plot( xr*ps, yr*ps, 's', color='green' )
                xrok.append(xr)
                yrok.append(yr)
            else:
                plt.plot( xr*ps, yr*ps, 'x', color='red' )
        if second_ring==True and Rr<Rlimit and Rr>6:
            if amp > 2.5e-4:
                plt.plot( xr*ps, yr*ps, 's', color='green' )
                xrok.append(xr)
                yrok.append(yr)
            else:
                plt.plot( xr, yr, 'x', color='red' )

    ring = np.vstack((xrok, yrok)).transpose()

    ell = EllipseModel()
    ell.estimate(ring)
    xc, yc, a, b, theta = ell.params
    boot_res=bootstrap_ellipse_fit(ring,10000)
    

    points=ell.predict_xy(np.linspace(0, 2 * np.pi, 50),params=(xc,yc,a,b,theta))
    sigma_2=np.sum(ell.residuals(ring)**2)
    ac,bc=a,b
    if a<b: 
        ac,bc=b,a
        print('reverse')
    cosi=bc/ac
    ecc=np.sqrt(1-bc*bc/ac/ac)
    angle=np.rad2deg(theta)
    if a<b:
        angle=angle+90
            
    print('Original xc=%.4f, yc=%.4f, a=%.4f, b=%.4f, theta=%.4f, i=%.4f' %(xc, yc, a*ps, b*ps, np.rad2deg(theta),np.rad2deg(np.arccos(cosi))))
        
    print('Bootstrap')
    print('xc=%.4f +- %.4f, 68 perc %.4f:%.4f, 95.4 perc %.4f:%.4f' % (boot_res['xc'][0],boot_res['xc'][1],boot_res['xc'][2][0],boot_res['xc'][2][1],boot_res['xc'][3][0],boot_res['xc'][3][1]))
    print('yc=%.4f +- %.4f, 68 perc %.4f:%.4f, 95.4 perc %.4f:%.4f' % (boot_res['yc'][0],boot_res['yc'][1],boot_res['yc'][2][0],boot_res['yc'][2][1],boot_res['yc'][3][0],boot_res['yc'][3][1]))
    print('a=%.4f +- %.4f, 68 perc %.4f:%.4f, 95.4 perc %.4f:%.4f' % (boot_res['a'][0]*ps,boot_res['a'][1]*ps,boot_res['a'][2][0]*ps,boot_res['a'][2][1]*ps,boot_res['a'][3][0]*ps,boot_res['a'][3][1]*ps))
    print('b=%.4f +- %.4f, 68 perc %.4f:%.4f, 95.4 perc %.4f:%.4f' % (boot_res['b'][0]*ps,boot_res['b'][1]*ps,boot_res['b'][2][0]*ps,boot_res['b'][2][1]*ps,boot_res['b'][3][0]*ps,boot_res['b'][3][1]*ps))
    print('theta=%.4f +- %.4f, 68 perc %.4f:%.4f, 95.4 perc %.4f:%.4f' % (np.rad2deg(boot_res['theta'][0]),(np.rad2deg(boot_res['theta'][1]+boot_res['theta'][0])-np.rad2deg(boot_res['theta'][0])),np.rad2deg(boot_res['theta'][2][0]),np.rad2deg(boot_res['theta'][2][1]),np.rad2deg(boot_res['theta'][3][0]),np.rad2deg(boot_res['theta'][3][1])))
    print('i=%.4f +- %.4f, 68 perc %.4f:%.4f, 95.4 perc %.4f:%.4f' % (np.rad2deg(np.arccos(boot_res['cosi'][0])),(np.rad2deg(np.arccos(boot_res['cosi'][0]+boot_res['cosi'][1]))-np.rad2deg(np.arccos(boot_res['cosi'][0]))),np.rad2deg(np.arccos(boot_res['cosi'][2][0])),np.rad2deg(np.arccos(boot_res['cosi'][2][1])),np.rad2deg(np.arccos(boot_res['cosi'][3][0])),np.rad2deg(np.arccos(boot_res['cosi'][3][1]))))
      

    plt.plot(points[:,0]*ps,points[:,1]*ps, color='cyan')
    plt.plot(xc*ps, yc*ps, '+', color='cyan')
    plt.title("\n".join(wrap(star+' ellipse gaussian fit for original values. a=%.2f, b=%.2f, i=%.2f' % (a, b,np.rad2deg(np.arccos(cosi))), 60)))
    f.northeast2(20,ps)
    
    plt.savefig(savefig1+ star+'_'+annulus+ "_ellipse_gauss_fit.jpeg",bbox_inches='tight', pad_inches=0.1)
    plt.close()  


    return  points, xc, yc, a, b, theta, cosi, ecc, sigma_2,boot_res

In [2]:

stars=['HR4049_20190108','HR4049_20190107',"HR4049_combined",'V709_Car','HR4226','01.SCI_UMon','01.SCI_AR_Pup']


#dirdat = '/media/kateryna/Data_Lin/PhD/IRDAP_reduction/kluska_0102.D-0696(A)/0-3_reduction/'
#stars = ['iras08544-4431_calib','hr4049','iras15469-5311','iras17038-4815','iw_car_calib','u_mon_2019-01-03_calib','u_mon_2019-01-14_calib','u_mon_combined','ru_cen_no_calib','ac_her']#['iras08544-4431_calib','hr4049','iras15469-5311','iras17038-4815','iw_car_calib','u_mon_2019-01-03_calib','u_mon_2019-01-14_calib','u_mon_combined']

#stars = ['iras08544-4431_calib']
fittype='PI'#'Q_phi'
bands=['V']

amp_dict={'HR4049_20190108':120,'HR4049_20190107':120,'HR4049_combined':120,'V709_Car':100,'HR4226':100,'01.SCI_UMon':100,'01.SCI_AR_Pup':100}
r0_dict={'HR4049_20190108':10,'HR4049_20190107':10,'HR4049_combined':10,'V709_Car':10,'HR4226':5,'01.SCI_UMon':8, '01.SCI_AR_Pup':10}
size_dict={'HR4049_20190108':10,'HR4049_20190107':10,'HR4049_combined':10,'V709_Car':10,'HR4226':5,'01.SCI_UMon':5, '01.SCI_AR_Pup':10}
star_names = {'01.SCI_UMon':'U Mon',"HR4049_combined":"HR4049_combined",'HD75885':'HD75885','01.SCI_AR_Pup':'AR_Pup','HR4049_20190108':'HR4049-2019-01-08','HR4049_20190107':'HR4049-2019-01-07','IRAS08544-4431':'IRAS08544-4431','UMon':'UMon','V709_Car':'V709_Car','HR4226':'HR4226'}

deproj=True
rad_bright=False

In [3]:
def create_dir(dirname):
    if not os.path.isdir(dirname):
        os.makedirs(dirname, exist_ok=True)

## Just bootstrap (to estimate significance of individual pixels, but does not give accurate uncertanty)

In [4]:
camera = 'ZIMPOL'

star=stars[-1]

dirdat0 = '/Users/aksitadeo/PycharmProjects/PythonProject/SPHERE_data/'+star+'/Unres+PSFcorr/'+camera+'/'
dirdat = '/Users/aksitadeo/PycharmProjects/PythonProject/SPHERE_data/'+star+'/Deconvolution_corr_tel+unres/'+camera+'/'

for annulus in bands:
    dir =dirdat +'deconvolved_'+fittype+'/'
    print(star)

    savefig1='/Users/aksitadeo/PycharmProjects/PythonProject/SPHERE_data/'+star+'/ellipse_with_gaussian_error/'+camera+'/'
    create_dir(savefig1)

    second_ring=False
    img_in, R, x, y, d_in= LoadImage(dir,annulus)# YOU NEED TO FILL THIS by loading the (deconvolved) SPHERE image
    Rlimit=30
    lim=35
    img = img_in * (R<Rlimit) * (R>1) ##mask in radius
    image_plot=np.arcsinh(img)
    n=img.shape[0]
    fig, ax = plt.subplots()
    
    ps=3.6

    d=d_in*ps
    plt.imshow(image_plot,vmax=np.max(image_plot),extent=(-d, d, d, -d))
    plt.xlim(-lim * ps, lim * ps)
    plt.ylim(-lim * ps, lim * ps)
    x0=(511.5-400)*ps-d
    y0=(511.5-400)*ps-d
    plt.plot(x0,y0,'+',color='white')
    plt.colorbar()    
    plt.title(star+' '+'original')
    plt.savefig(savefig1+ star+'_'+annulus+ "_data_for_fit.jpeg",bbox_inches='tight', pad_inches=0.1)    
    plt.close()  

    points, xc, yc, a, b, theta, cosi, ecc, sigma_2,bootstrap=gauss_ell_fit(img,x,y,ps,savefig1,star,annulus,Rlimit)

    print(xc, yc, a, b, theta,str(np.rad2deg(np.arccos(cosi))),ecc)

    logfile=open(savefig1+star+'_'+annulus+'_ellipse_gauss_fit.txt','w+')
    logfile.writelines(star+'\n')
    logfile.writelines('Annulus for stellar polarisation '+annulus+'\n')
    logfile.writelines("center = (%.4f , %.4f) \n" % (xc*ps, yc*ps))
    logfile.writelines("angle of rotation (rad) = %.4f \n" % theta)
    logfile.writelines("half axes im mas= %.4f, %.4f \n" % (a*ps,b*ps))
    logfile.writelines("sigma^2 = %f \n" % sigma_2)
    angle=np.rad2deg(theta)-90
    logfile.writelines('Inclination for the deprojection '+str(np.rad2deg(np.arccos(cosi)))+'\n')
    logfile.writelines('PA (theta-90) (deg)'+str(angle)+'\n')
    logfile.writelines('Eccentricity '+str(ecc)+'\n')
    logfile.writelines('Bootstrap '+'\n')
    logfile.writelines("center = (%.4f +- %.4f, %.4f +- %.4f) \n" % (bootstrap['xc'][0]*ps,bootstrap['xc'][1]*ps,bootstrap['yc'][0]*ps,bootstrap['yc'][1]*ps))
    logfile.writelines("half axes im mas= %.4f+- %.4f, %.4f+- %.4f \n" % (bootstrap['a'][0]*ps,bootstrap['a'][1]*ps,bootstrap['b'][0]*ps,bootstrap['b'][1]*ps))
    logfile.writelines("theta = %.4f +- %.4f \n" % (np.rad2deg(bootstrap['theta'][0]),np.rad2deg(bootstrap['theta'][1])))
    logfile.writelines("i = %.4f +- %.4f \n" % (np.rad2deg(np.arccos(bootstrap['cosi'][0])),(np.rad2deg(np.arccos(bootstrap['cosi'][0]+bootstrap['cosi'][1]))-np.rad2deg(np.arccos(bootstrap['cosi'][0])))))
   
    logfile.writelines("68% interval \n")
    logfile.writelines("a = %.4f %.4f \n" % (bootstrap['a'][2][0]*ps,bootstrap['a'][2][1]*ps))
    logfile.writelines("b = %.4f %.4f \n"% (bootstrap['b'][2][0]*ps,bootstrap['b'][2][1]*ps))
    logfile.writelines("theta (deg) = %.4f %.4f \n" % (np.rad2deg(bootstrap['theta'][2][0]),np.rad2deg(bootstrap['theta'][2][1])))
    logfile.writelines("i (deg)= %.4f %.4f \n" % (np.rad2deg(np.arccos(bootstrap['cosi'][2][0])),np.rad2deg(np.arccos(bootstrap['cosi'][2][1]))))
    
    logfile.writelines("95% interval \n")
    logfile.writelines("a = %.4f %.4f \n" % (bootstrap['a'][3][0]*ps,bootstrap['a'][3][1]*ps))
    logfile.writelines("b = %.4f %.4f \n"% (bootstrap['b'][3][0]*ps,bootstrap['b'][3][1]*ps))
    logfile.writelines("theta (deg) = %.4f %.4f \n" % (np.rad2deg(bootstrap['theta'][3][0]),np.rad2deg(bootstrap['theta'][3][1])))
    logfile.writelines("i (deg) = %.4f %.4f \n" % (np.rad2deg(np.arccos(bootstrap['cosi'][3][0])),np.rad2deg(np.arccos(bootstrap['cosi'][3][1]))))
    

    logfile.writelines('\n')
    logfile.close()

01.SCI_AR_Pup
Original xc=-7.9931, yc=0.5994, a=56.0004, b=26.3724, theta=174.5620, i=61.9051
Bootstrap
xc=-7.9874 +- 0.5638, 68 perc -8.5520:-7.4281, 95.4 perc -9.0144:-6.7779
yc=0.6257 +- 0.1344, 68 perc 0.4922:0.7587, 95.4 perc 0.3558:0.8886
a=56.2892 +- 2.8517, 68 perc 53.4274:59.3534, 95.4 perc 50.5878:61.4744
b=26.2484 +- 0.8981, 68 perc 25.3158:27.1915, 95.4 perc 24.6996:28.1262
theta=174.3208 +- 2.8676, 68 perc 172.9562:175.7760, 95.4 perc 171.5912:177.4718
i=62.0985 +- -2.3073, 68 perc 64.6214:59.7029, 95.4 perc 65.5917:57.2226
-7.9930691631249084 0.5994393221874403 15.555672918844706 7.325677602585374 3.046681250051385 61.90513217806808 0.8821690526168223


## With scaling of the ellipse (see more details in papers, this is final uncertanty estimation)

In [5]:
def error_estimation_using_ellipse_scalling(datapoints, xc, yc, a, b, theta,std, nforstd,reverse,band):
    for_a=range(50)
    
    savefigfits='/Users/aksitadeo/PycharmProjects/PythonProject/SPHERE_data/'+star+'/scaling_ellipse_with_gaussian/'+camera+'/'
    create_dir(savefigfits)
    
    if reverse:
        major_axis=b
        theta=theta+np.deg2rad(90)
    else:
        major_axis=a
    x_data = datapoints[:, 0]
    y_data = datapoints[:, 1]
    stepa=0.02*major_axis
    stepb=0.05
    len_b=int((1.5*major_axis*0.6)//stepb)    

    std_array=np.ones((len(for_a), len_b,30))*1000
    i_a=np.ones((len(for_a), len_b,30))*10000
    i_b=np.ones((len(for_a), len_b,30))*1000
    i_array=np.ones((len(for_a), len_b,30))*1000
    theta_array=np.ones((len(for_a), len_b,30))*1000
    
    for it_for_a in for_a:
        a_it=0.5*major_axis+it_for_a*stepa
        print(it_for_a)
        for it_for_b in range(0,int(((0.6*a_it)//stepb))):
            for i_the in range(30):
                i_theta=theta+np.deg2rad(-30+2*i_the)
                b_it=0.4*a_it+ it_for_b*stepb
                
                ell_in = EllipseModel()
                ell = EllipseModel()
               
                points=ell_in.predict_xy(np.linspace(0* np.pi, 2 * np.pi, 50),params=(xc,yc,a_it,b_it,i_theta))           #to create initial elipse points with setted parameters 
                
                ell.estimate(points) #to establish new ellipse with seted parameters
                xc_, yc_, a_, b_, theta_ = ell.params #to check parameters
                points_=ell.predict_xy(np.linspace(0* np.pi, 2 * np.pi, 50),params=(xc_, yc_, a_, b_, theta_)) # to plot this ellipse after
            

               
                a_end, b_end=a_, b_
                if b_>=a_:
                    a_end, b_end=b_,a_
                cosi=b_end/a_end
                incl= np.rad2deg(np.arccos(cosi))
                ecc=np.sqrt(1-b_end**2/a_end**2)
                
                sigma_2=np.sum(ell.residuals(datapoints)**2) #to find residials with ring points
               
                pa=np.rad2deg(i_theta)
                if reverse:
                    if np.rad2deg(i_theta)<90:
                        pa=pa+90
                    else:
                        pa=pa-90
                std_array[it_for_a,it_for_b,i_the]=sigma_2
                i_b[it_for_a,it_for_b,i_the]=b_it
                i_a[it_for_a,it_for_b,i_the]=a_it
                i_array[it_for_a,it_for_b,i_the]=incl
                theta_array[it_for_a,it_for_b,i_the]=pa

                #fig, axs = plt.subplots(1, 1, sharex=True, sharey=True)
                #axs.scatter(x_data, y_data)
                #axs.plot(xc_, yc_, "+", color='red')
                #plt.plot(points_[:,0],points_[:,1], color='red')
                #plt.title(star+' '+fittype)
                #plt.savefig(savefigfits+str(it_for_a)+ str(it_for_b)+str(i_the)+'_arc.jpeg',bbox_inches='tight', pad_inches=0.1)
                
                #plt.close()
            


    print('initial std'+str(std))
    print('best from array'+str(np.min(std_array)))
    

    i_b[i_b==1000]=np.nan
    i_a[i_a==1000]=np.nan
    i_array[i_array==1000]=np.nan
    theta_array[theta_array==1000]=np.nan
    index=np.where(std_array == np.nanmin(std_array))
    print(index)
    print(index[0][0])
    plt.imshow(std_array[:,:,index[2][0]],vmax=10*std)
    
    #plt.xlim(np.nanmin(i_b),np.nanmax(i_b))
    #plt.ylim(np.nanmax(i_a),np.nanmin(i_a))
    plt.xlabel('b, iteration')
    plt.ylabel('a, iteration')
    plt.colorbar()

    plt.title(star_names[star])
    plt.savefig(savefigfits+star+'_'+band+'_std.png',bbox_inches='tight', pad_inches=0.1)
    plt.close()
    
    print(std_array[std_array == np.min(std_array)], i_a[std_array == np.min(std_array)], i_b[std_array == np.min(std_array)],i_array[std_array == np.min(std_array)],np.rad2deg(theta_array[std_array == np.min(std_array)]))

    b_low=np.nanmin(i_b[std_array[:,:,index[2][0]]<=nforstd*std])
    b_up=np.nanmax(i_b[std_array[:,:,index[2][0]]<=nforstd*std])
    a_low=np.nanmin(i_a[std_array[:,:,index[2][0]]<=nforstd*std])
    a_up=np.nanmax(i_a[std_array[:,:,index[2][0]]<=nforstd*std])


    incl_low=np.nanmin(i_array[std_array[:,:,index[2][0]]<=nforstd*std])
    incl_up=np.nanmax(i_array[std_array[:,:,index[2][0]]<=nforstd*std])
    theta_low=np.nanmin(theta_array[std_array[:,:,:]<=nforstd*std])
    theta_up=np.nanmax(theta_array[std_array[:,:,:]<=nforstd*std])
    
    print('a range')
    print(a_low,a_up)
    print('b range')
    print(b_low,b_up)
    print('inclination range')
    print(incl_low,incl_up)

    b_low5=np.nanmin(i_b[std_array[:,:,index[2][0]]<=2*std])
    b_up5=np.nanmax(i_b[std_array[:,:,index[2][0]]<=2*std])
    a_low5=np.nanmin(i_a[std_array[:,:,index[2][0]]<=2*std])
    a_up5=np.nanmax(i_a[std_array[:,:,index[2][0]]<=2*std])


    incl_low5=np.nanmin(i_array[std_array[:,:,index[2][0]]<=2*std])
    incl_up5=np.nanmax(i_array[std_array[:,:,index[2][0]]<=2*std])
    theta_low5=np.nanmin(theta_array[std_array[:,:,:]<=2*std])
    theta_up5=np.nanmax(theta_array[std_array[:,:,:]<=2*std])
    
    print('a range 2 sigma')
    print(a_low5,a_up5)
    print('b range 2 sigma')
    print(b_low5,b_up5)
    print('inclination range 2 sigma')
    print(incl_low5,incl_up5)
    
    return a_low,a_up, b_low, b_up, incl_low,incl_up, std_array[std_array == np.min(std_array)], theta_array[std_array == np.min(std_array)],i_a[std_array == np.min(std_array)], i_b[std_array == np.min(std_array)],i_array[std_array == np.min(std_array)],a_low5,a_up5, b_low5, b_up5, incl_low5,incl_up5, theta_low,theta_up,theta_low5,theta_up5
            
def gauss_ell_fit(img,x,y,ps,savefig, star,annulus,Rlimit):
    n=img.shape[0]
    shift=n/2-0.5

    rad, theta, amps = GetRingCoord(img, x, y, nangles = 360)

    d=n*ps/2
    fig, ax = plt.subplots()
    plt.imshow(np.arcsinh(img), extent=[-d, d, d, -d])
    plt.xlim(-lim * ps, lim * ps)
    plt.ylim(-lim * ps, lim * ps)
    plt.colorbar()
    xring = rad*np.cos(theta*np.pi/180.)
    yring = rad*np.sin(theta*np.pi/180.)

    #print(amps)

    xrok, yrok = [], []

    for xr, yr, amp in zip(xring, yring, amps):
        Rr = np.sqrt(xr**2+yr**2)
        if Rr<Rlimit and second_ring==False:
            if amp > 2.5e-4:
                plt.plot( xr*ps, yr*ps, 's', color='green' )
                xrok.append(xr)
                yrok.append(yr)
            else:
                plt.plot( xr*ps, yr*ps, 'x', color='red' )
        if second_ring==True and Rr<Rlimit and Rr>6:
            if amp > 2.5e-4:
                plt.plot( xr*ps, yr*ps, 's', color='green' )
                xrok.append(xr)
                yrok.append(yr)
            else:
                plt.plot( xr, yr, 'x', color='red' )

    ring = np.vstack((xrok, yrok)).transpose()

    ell = EllipseModel()
    ell.estimate(ring)
    xc, yc, a, b, theta = ell.params
    boot_res=bootstrap_ellipse_fit(ring,1000)
    

    points=ell.predict_xy(np.linspace(0, 2 * np.pi, 50),params=(xc,yc,a,b,theta))
    sigma_2=np.sum(ell.residuals(ring)**2)
    ac,bc=a,b
    reverse=False
    if a<b: 
        ac,bc=b,a
        print('reverse')
        reverse=True
    cosi=bc/ac
    ecc=np.sqrt(1-bc*bc/ac/ac)
    angle=np.rad2deg(theta)
    if a<b:
        angle=angle+90
            
    #print('Original xc=%.4f, yc=%.4f, a=%.4f, b=%.4f, theta=%.4f, i=%.4f' %(xc, yc, a*ps, b*ps, np.rad2deg(theta),np.rad2deg(np.arccos(cosi))))
        
    #print('Bootstrap')
    #print('xc=%.4f +- %.4f, 68 perc %.4f:%.4f, 95.4 perc %.4f:%.4f' % (boot_res[0][0],boot_res[0][1],boot_res[0][2][0],boot_res[0][2][1],boot_res[0][3][0],boot_res[0][3][1]))
    #print('yc=%.4f +- %.4f, 68 perc %.4f:%.4f, 95.4 perc %.4f:%.4f' % (boot_res[1][0],boot_res[1][1],boot_res[1][2][0],boot_res[1][2][1],boot_res[1][3][0],boot_res[1][3][1]))
    #print('a=%.4f +- %.4f, 68 perc %.4f:%.4f, 95.4 perc %.4f:%.4f' % (boot_res[2][0]*ps,boot_res[2][1]*ps,boot_res[2][2][0]*ps,boot_res[2][2][1]*ps,boot_res[2][3][0]*ps,boot_res[2][3][1]*ps))
    #print('b=%.4f +- %.4f, 68 perc %.4f:%.4f, 95.4 perc %.4f:%.4f' % (boot_res[3][0]*ps,boot_res[3][1]*ps,boot_res[3][2][0]*ps,boot_res[3][2][1]*ps,boot_res[3][3][0]*ps,boot_res[3][3][1]*ps))
    #print('theta=%.4f +- %.4f, 68 perc %.4f:%.4f, 95.4 perc %.4f:%.4f' % (np.rad2deg(boot_res[4][0]),(np.rad2deg(boot_res[4][1]+boot_res[4][0])-np.rad2deg(boot_res[4][0])),np.rad2deg(boot_res[4][2][0]),np.rad2deg(boot_res[4][2][1]),np.rad2deg(boot_res[4][3][0]),np.rad2deg(boot_res[4][3][1])))
    #print('i=%.4f +- %.4f, 68 perc %.4f:%.4f, 95.4 perc %.4f:%.4f' % (np.rad2deg(np.arccos(boot_res[5][0])),(np.rad2deg(np.arccos(boot_res[5][0]+boot_res[5][1]))-np.rad2deg(np.arccos(boot_res[5][0]))),np.rad2deg(np.arccos(boot_res[5][2][0])),np.rad2deg(np.arccos(boot_res[5][2][1])),np.rad2deg(np.arccos(boot_res[5][3][0])),np.rad2deg(np.arccos(boot_res[5][3][1]))))
      
    

    plt.plot(points[:,0]*ps,points[:,1]*ps, color='cyan')
    plt.plot(xc*ps, yc*ps, '+', color='cyan')
    plt.title("\n".join(wrap(star+' ellipse gaussian fit for original values. a=%.2f, b=%.2f, i=%.2f' % (a, b,np.rad2deg(np.arccos(cosi))), 60)))
    f.northeast2(20,ps)
    if second_ring:
        plt.savefig(savefig+ star+'_'+annulus+ "_ellipse_gauss_fit_2.jpeg",bbox_inches='tight', pad_inches=0.1)
        
    else:
        plt.savefig(savefig1+ star+'_'+annulus+ "_ellipse_gauss_fit.jpeg",bbox_inches='tight', pad_inches=0.1)
        
    plt.close()  
    
    return  points, xc, yc, a, b, theta, cosi, ecc, sigma_2,boot_res, ring, reverse


In [6]:

for annulus in bands:
    dir =dirdat +'deconvolved_'+fittype+'/'
    print(star)


    # savefig1='/media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/'+'scaling_ellipse_with_gaussian/'
    savefigfits='/Users/aksitadeo/PycharmProjects/PythonProject/SPHERE_data/'+star+'/scaling_ellipse_with_gaussian/'+camera+'/'
    create_dir(savefigfits)

    second_ring=False
    img_in, R, x, y, d_in= LoadImage(dir,annulus)# YOU NEED TO FILL THIS by loading the (deconvolved) SPHERE image
    Rlimit=50
    lim=35
    img = img_in * (R<Rlimit) * (R>1) ##mask in radius
    image_plot=np.arcsinh(img)
    n=img.shape[0]
    fig, ax = plt.subplots()
    
    ps=12.27
    d=d_in*ps
    plt.imshow(image_plot,vmax=np.max(image_plot),extent=(-d, d, d, -d))
    plt.xlim(-lim * ps, lim * ps)
    plt.ylim(-lim * ps, lim * ps)
    x0=(511.5-400)*ps-d
    y0=(511.5-400)*ps-d
    plt.plot(x0,y0,'+',color='white')
    plt.colorbar()    
    plt.title(star+' '+'original')
    plt.savefig(savefig1+ star+'_'+annulus+ "_data_for_fit.jpeg",bbox_inches='tight', pad_inches=0.1)    
    plt.close()  

    points, xc, yc, a, b, theta, cosi, ecc, sigma_2,bootstrap,ring,reverse=gauss_ell_fit(img,x,y,ps,savefig1,star,annulus,Rlimit)
    a_low,a_up, b_low, b_up, incl_low,incl_up, best_std, best_theta,best_a, best_b, best_i, a_low5,a_up5, b_low5, b_up5, incl_low5,incl_up5,theta_low,theta_up,theta_low5,theta_up5=error_estimation_using_ellipse_scalling(ring, xc, yc, a, b, theta,sigma_2, 3,reverse,annulus)
    if reverse:
        a_forerror,b_forerror=b*ps,a*ps
        theta_forerror=np.rad2deg(theta)+90
    else:
        a_forerror,b_forerror=a*ps,b*ps
        theta_forerror=np.rad2deg(theta)
    i_forerror=np.rad2deg(np.arccos(cosi))
    theta_for_result=theta_forerror

    if theta_forerror<0:
        theta_for_result=theta_forerror+180
   
    print(xc, yc, a, b, theta,str(np.rad2deg(np.arccos(cosi))),ecc)

    logfile=open(savefig1+star+'_'+annulus+'_ellipse_gauss_fit.txt','w+')
    logfile.writelines(star+'\n')
    logfile.writelines('Annulus for stellar polarisation '+annulus+'\n')
    logfile.writelines("center = (%.4f , %.4f) \n" % (xc*ps, yc*ps))
    logfile.writelines("angle of rotation (rad) = %.4f \n" % theta)
    logfile.writelines("half axes im mas= %.4f, %.4f \n" % (a_forerror,b_forerror))
    logfile.writelines("sigma^2 = %f \n" % sigma_2)

    logfile.writelines('Inclination for the deprojection '+str(np.rad2deg(np.arccos(cosi)))+'\n')
    logfile.writelines('PA (theta) (deg)'+str(theta_for_result)+'\n')
    logfile.writelines('Eccentricity '+str(ecc)+'\n')
    logfile.writelines('Bootstrap '+'\n')
    logfile.writelines("center = (%.4f +- %.4f, %.4f +- %.4f) \n" % (bootstrap['xc'][0]*ps,bootstrap['xc'][1]*ps,bootstrap['yc'][0]*ps,bootstrap['yc'][1]*ps))
    logfile.writelines("half axes im mas= %.4f+- %.4f, %.4f+- %.4f \n" % (bootstrap['a'][0]*ps,bootstrap['a'][1]*ps,bootstrap['b'][0]*ps,bootstrap['b'][1]*ps))
    logfile.writelines("theta = %.4f +- %.4f \n" % (np.rad2deg(bootstrap['theta'][0]),np.rad2deg(bootstrap['theta'][1])))
    logfile.writelines("i = %.4f +- %.4f \n" % (np.rad2deg(np.arccos(bootstrap['cosi'][0])),(np.rad2deg(np.arccos(bootstrap['cosi'][0]+bootstrap['cosi'][1]))-np.rad2deg(np.arccos(bootstrap['cosi'][0])))))
   
    logfile.writelines("68% interval \n")
    logfile.writelines("a = %.4f %.4f \n" % (bootstrap['a'][2][0]*ps,bootstrap['a'][2][1]*ps))
    logfile.writelines("b = %.4f %.4f \n"% (bootstrap['b'][2][0]*ps,bootstrap['b'][2][1]*ps))
    logfile.writelines("theta (deg) = %.4f %.4f \n" % (np.rad2deg(bootstrap['theta'][2][0]),np.rad2deg(bootstrap['theta'][2][1])))
    logfile.writelines("i (deg)= %.4f %.4f \n" % (np.rad2deg(np.arccos(bootstrap['cosi'][2][0])),np.rad2deg(np.arccos(bootstrap['cosi'][2][1]))))
    
    logfile.writelines("95% interval \n")
    logfile.writelines("a = %.4f %.4f \n" % (bootstrap['a'][3][0]*ps,bootstrap['a'][3][1]*ps))
    logfile.writelines("b = %.4f %.4f \n"% (bootstrap['b'][3][0]*ps,bootstrap['b'][3][1]*ps))
    logfile.writelines("theta (deg) = %.4f %.4f \n" % (np.rad2deg(bootstrap['theta'][3][0]),np.rad2deg(bootstrap['theta'][3][1])))
    logfile.writelines("i (deg) = %.4f %.4f \n" % (np.rad2deg(np.arccos(bootstrap['cosi'][3][0])),np.rad2deg(np.arccos(bootstrap['cosi'][3][1]))))
    
    logfile.writelines("Ranges for parameters by changing a,b,theta and comparing std. Upper limit is 3*sigma^2 \n")
    logfile.writelines("best a = %.4f  \n" % (best_a*ps)) 
    logfile.writelines("best b = %.4f  \n" % (best_b*ps))
    logfile.writelines("best i (deg) = %.4f  \n" % (best_i))
    logfile.writelines("best theta (deg) = %.4f  \n" % (best_theta)) 
    logfile.writelines("best std = %.4f  \n" % (best_std))    
    logfile.writelines("a = %.4f %.4f  (+ %.4f - %.4f)\n" % (a_low*ps,a_up*ps, a_up*ps-a_forerror,a_forerror-a_low*ps))
    logfile.writelines("b = %.4f %.4f (+ %.4f - %.4f)\n"% (b_low*ps,b_up*ps, b_up*ps-b_forerror,b_forerror-b_low*ps))
    logfile.writelines("i (deg) = %.4f %.4f (+ %.4f - %.4f)\n" % (incl_low,incl_up, incl_up-i_forerror,i_forerror-incl_low))
    logfile.writelines("theta (deg) = %.4f %.4f (+ %.4f - %.4f)\n" % (theta_low,theta_up, theta_up-theta_forerror,theta_forerror-theta_low))

    logfile.writelines("Ranges for parameters by changing a,b,theta and comparing std. Upper limit is 2*sigma^2 \n")
    logfile.writelines("a = %.4f %.4f (+ %.4f - %.4f)\n" % (a_low5*ps,a_up5*ps, a_up5*ps-a_forerror, a_forerror-a_low5*ps))
    logfile.writelines("b = %.4f %.4f (+ %.4f - %.4f)\n"% (b_low5*ps,b_up5*ps, b_up5*ps-b_forerror, b_forerror-b_low5*ps))
    logfile.writelines("i (deg) = %.4f %.4f (+ %.4f - %.4f)\n" % (incl_low5,incl_up5, incl_up5-i_forerror, i_forerror-incl_low5))
    logfile.writelines("theta (deg) = %.4f %.4f (+ %.4f - %.4f)\n" % (theta_low5,theta_up5, theta_up5-theta_forerror,theta_forerror-theta_low5))

    logfile.writelines('\n')
    logfile.close()


01.SCI_AR_Pup
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
initial std4403.720017795966
best from array1000.0
(array([ 0,  0,  0, ..., 49, 49, 49], shape=(144870,)), array([ 94,  94,  94, ..., 283, 283, 283], shape=(144870,)), array([ 0,  1,  2, ..., 27, 28, 29], shape=(144870,)))
0
[1000. 1000. 1000. ... 1000. 1000. 1000.] [10000. 10000. 10000. ... 10000. 10000. 10000.] [nan nan nan ... nan nan nan] [nan nan nan ... nan nan nan] [nan nan nan ... nan nan nan]
a range
8.858729549249201 10000.0
b range
4.4293647746246005 17.033877080849386
inclination range
4.4201841994863225 66.42182152179828
a range 2 sigma
11.0734119365615 10000.0
b range 2 sigma
5.415237729549521 15.024557405199454
inclination range 2 sigma
4.978388887837734 66.42182152179828
-10.030350722435838 1.0531791608914611 15.819159909373573 9.427007763196835 0.06801135263126312 53.42151185587451 0.8030412728483756


TypeError: only length-1 arrays can be converted to Python scalars